reference https://www.blakeporterneuro.com/learning-python-project-3-scrapping-data-from-steams-community-market/
this is for scrapping market place of a particular game which is technacally not what I want as I am interested more in the payer count

video that helped me espicially in the inital scrapping of review to understanad the workflow of scrapping and acessing data https://www.youtube.com/watch?v=Id2iYV3EfG4

# 1. Web Scrapping

a) review scrapping




In [29]:
# each steam page has a unqiue game id in their url so this could help 

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
import random

options = Options()


# game id
game_id = 413150 

# 413150  Stardew Valley
# 1623730 PalWorld

#url_template
game_url = f"https://steamcommunity.com/app/{game_id}/reviews/?p=1&browsefilter=trendyear&filterLanguage=english"
print(game_url)


# https://www.selenium.dev/selenium/docs/api/rb/Selenium/WebDriver/Chrome/Options.html
#https://groups.google.com/g/robotframework-users/c/r3rxgSv1Cs0
# https://www.selenium.dev/documentation/webdriver/getting_started/first_script/    instruction

options = Options()
language = "en-US"
options.add_argument(f"--lang={language}")

driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(game_url)



https://steamcommunity.com/app/413150/reviews/?p=1&browsefilter=trendyear&filterLanguage=english


In [ ]:
# driver.quit()

## Handy web scrape functions: 
https://www.geeksforgeeks.org/execute_script-driver-method-selenium-python/
https://developer.mozilla.org/en-US/docs/Web/API/Window

Most Browser have a windows as a alias to the tab


In [31]:
def get_Y_scroll_position(driver):
    return driver.execute_script("return window.pageYOffset")


# https://stackoverflow.com/questions/42982950/how-to-scroll-down-the-page-till-bottomend-page-in-the-selenium-webdriver

def scroll_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(random.uniform(1, 3))

    


# what do I want?
# Recommended or not/hours played/actual review/ early acess or not
def get_review_data(card, game_id):     
    # Retrieve the Steam ID (or "Private ID" if not a valid digit)
    steam_id = get_steam_id(card)
    
    # Retrieve the number of hours played
    game_time = get_hour(card)
    
    # Retrieve the recommendation status (e.g., Recommended or Not)
    recommend = get_recommendation(card)
    
    # Retrieve the helpful review count
    helpful_count = get_helpful(card)
    
    # Retrieve the date the review was posted
    review_date = get_date(card)
    
    # Retrieve the comment or review text
    review_comment = get_comment(card)
    
    # Retrieve whether the game is early access or fully released
    early_access = get_early_access(card)
    
    refund = get_refund(card)
    
    # Return all the collected data in a dictionary or tuple
    return {
        "steam_id": steam_id,
        "game_time": game_time,
        "recommend": recommend,
        "helpful_count": helpful_count,
        "review_date": review_date,
        "review_comment": review_comment,
        "early_access": early_access,
        "refund": refund,
        "game_id": game_id
        
    }
    

        
        
    
    
    



# https://www.browserstack.com/guide/find-element-by-xpath-in-selenium
# https://selenium-python.readthedocs.io/locating-elements.html
 # the elments are index by 1 start
from selenium.common.exceptions import NoSuchElementException

def get_steam_id(card):
    try:
        profile = card.find_element(By.XPATH, './/div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')
        id = profile.split("/")[-2]
        if id.isdigit():
            return int(id)
        else:
            return "Private ID" # Return empty string if the id is not a digit
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_hour(card):
    try:
        return card.find_element(By.CLASS_NAME, "hours").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_recommendation(card):
    try:
        return card.find_element(By.CLASS_NAME, "title").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_helpful(card):
    try:
        return card.find_element(By.CLASS_NAME, "found_helpful").text.split("\n")[0]
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_date(card):
    try:
        return card.find_element(By.CLASS_NAME, "date_posted").text
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_comment(card):
    try:
        text = card.find_element(By.XPATH, './/div[@class ="apphub_CardTextContent"]').text
        lines = text.split("\n")[-1]  # Assuming you want the last line
        return lines
    except NoSuchElementException:
        return "Not Found"  # Return empty string if element is not found

def get_early_access(card):
    try:
        early_access = card.find_element(By.CLASS_NAME, "early_access_review").text
        return early_access
    except NoSuchElementException:
        return "Full_Release"  # Default value if element is not found
    
def get_refund(card):
    try:
        early_access = card.find_element(By.CLASS_NAME, "refunded").text
        return early_access
    except NoSuchElementException:
        return "not refunded"  # Default value if element is not found





# Test for actually getting review card
review_cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")  # Returns a list

print(review_cards)

if len(review_cards) > 2:  # Ensure there are enough elements
    print(get_steam_id(review_cards[7]))  # Pass a single element
else:
    print("Not enough review cards found.")
    
print(get_hour(review_cards[4]))
print(get_helpful(review_cards[1]))
print(get_date(review_cards[1]))
print(get_comment(review_cards[5]))
print(get_early_access(review_cards[9]))

# print(driver.find_elements(By.CLASS_NAME, "hours")[1].text) #change web element into text
    

[<selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1d3", element="f.85553C4BB2F94795880F01564C0B70A3.d.500360846815F3AFB960132775604F45.e.52")>, <selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1d3", element="f.85553C4BB2F94795880F01564C0B70A3.d.500360846815F3AFB960132775604F45.e.53")>, <selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1d3", element="f.85553C4BB2F94795880F01564C0B70A3.d.500360846815F3AFB960132775604F45.e.54")>, <selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1d3", element="f.85553C4BB2F94795880F01564C0B70A3.d.500360846815F3AFB960132775604F45.e.55")>, <selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1d3", element="f.85553C4BB2F94795880F01564C0B70A3.d.500360846815F3AFB960132775604F45.e.56")>, <selenium.webdriver.remote.webelement.WebElement (session="eb533b750e440be5cd8af6aeb0f0b1

In [ ]:
def get_Y_scroll_position(driver):
    return driver.execute_script("return window.pageYOffset")


# https://stackoverflow.com/questions/42982950/how-to-scroll-down-the-page-till-bottomend-page-in-the-selenium-webdriver

def scroll_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1)
    


# single game scrapping 
reviews = []
id_set = set()
max_scroll = 10


try:
    previous_position = get_Y_scroll_position(driver)
    state = True
    scroll_count = 0
    while(state):
        element = driver.find_elements(By.CLASS_NAME, "apphub_Card")
        
        for card in element:
            steam_id = get_steam_id(card)         
            if steam_id in id_set:
                continue
            else:
                review_package = get_review_data(card, game_id)
                reviews.append(review_package)
                id_set.add(steam_id)
                
            
        while(scroll_count < max_scroll):
            scroll_bottom(driver)
            current_position = get_Y_scroll_position(driver)
            
            # This means we reached the end of the scrolling page
            if(previous_position == current_position):
                scroll_count+=1
                # safer random break less likly to get blocked
                time.sleep(random.uniform(1, 3))
                
                # give the program a bit slack maybe its the loading time issue
                if(scroll_count>max_scroll/2):
                   state = False
                   break
            else:
                previous_position = current_position
                break
                
           
            
            
             
            # limit one comment per id, this is interesting cause some of the user would turn privacy on and the id is actually blank so beware 
            # perhaps not such a big deal since all such privacy on users would have the string "id" as their id hence appering once as an edge case
    
    
except Exception as e:
    # Handles the exception
    print(f"An error occurred: {e}")

# finally:
#    driver.quit()
     

## data loading


In [33]:
import pandas as pd

if reviews:
    # Convert the list of dictionaries (reviews) into a pandas DataFrame
    reviews_df = pd.DataFrame(reviews)
    
    # Save the DataFrame to a CSV file
    reviews_df.to_csv("reviews.csv", index=False)
    print("Reviews have been successfully saved to 'reviews.csv'.")
else:
    print("No reviews were collected.")
    
print(reviews_df)

Reviews have been successfully saved to 'reviews.csv'.
              steam_id            game_time    recommend  \
0           Private ID  507.4 hrs on record  Recommended   
1    76561199208930069  127.8 hrs on record  Recommended   
2    76561198836793817  158.2 hrs on record  Recommended   
3    76561198111477632  255.6 hrs on record  Recommended   
4    76561199235027626  195.9 hrs on record  Recommended   
..                 ...                  ...          ...   
541  76561199472699914  170.6 hrs on record  Recommended   
542  76561198965037783   81.4 hrs on record  Recommended   
543  76561198138740939    8.5 hrs on record  Recommended   
544  76561198290841144  230.7 hrs on record  Recommended   
545  76561198855112335  838.7 hrs on record  Recommended   

                              helpful_count               review_date  \
0    1,240 people found this review helpful    Posted: March 16, 2024   
1      535 people found this review helpful      Posted: May 12, 2024   
2    

# Scraping player count data

{'Current Players': '42764', 'Peak Players 24h': '90634', 'Peak Players All Time': '236614'}
